In [1]:
!pip install tensorflow_text

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 26.7 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.2.1
    Uninstalling keras-3.2.1:
      Successfully uninstalled keras-3.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.


2024-05-21 12:25:31.709264: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 12:25:31.709355: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 12:25:31.883255: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd

df = pd.read_csv("/kaggle/input/spam-dataset1/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [4]:
df['Spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df.head()

,Category,Message,Spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [5]:
df_spam=df[df['Spam']==1]
df_spam.shape

(747, 3)

In [6]:
df['Spam'].value_counts()

Spam
0    4825
1     747
Name: count, dtype: int64

In [7]:
df_ham=df[df['Spam']==0]
df_ham.shape

(4825, 3)

In [8]:
df_ham=df_ham.sample(df_spam.shape[0])
df_ham.shape

(747, 3)

In [9]:
df=pd.concat([df_ham,df_spam],axis=0)
df.shape

(1494, 3)

In [10]:
df

,Category,Message,Spam
1579,ham,How to Make a girl Happy? It's not at all diff...,0
2385,ham,"Sorry, I'll call later",0
530,ham,Jay says that you're a double-faggot,0
4025,ham,Wat time ü wan today?,0
5421,ham,"Hi elaine, is today's meeting confirmed?",0
...,...,...,...
5537,spam,Want explicit SEX in 30 secs? Ring 02073162414...,1
5540,spam,ASKED 3MOBILE IF 0870 CHATLINES INCLU IN FREE ...,1
5547,spam,Had your contract mobile 11 Mnths? Latest Moto...,1
5566,spam,REMINDER FROM O2: To get 2.50 pounds free call...,1


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['Message'], df['Spam'], test_size=0.2, random_state=42)

In [12]:
X_train.tolist()

['Does daddy have a bb now.',
 'Did you get any gift? This year i didnt get anything. So bad',
 'Hello darling how are you today? I would love to have a chat, why dont you tell me what you look like and what you are in to sexy?',
 'Somebody set up a website where you can play hold em using eve online spacebucks',
 'Reply with your name and address and YOU WILL RECEIVE BY POST a weeks completely free accommodation at various global locations www.phb1.com ph:08700435505150p',
 "URGENT! Last weekend's draw shows that you have won £1000 cash or a Spanish holiday! CALL NOW 09050000332 to claim. T&C: RSTM, SW7 3SS. 150ppm",
 'Ur cash-balance is currently 500 pounds - to maximize ur cash-in now send GO to 86688 only 150p/meg. CC: 08718720201 HG/Suite342/2lands Row/W1j6HL',
 'Cashbin.co.uk (Get lots of cash this weekend!) www.cashbin.co.uk Dear Welcome to the weekend We have got our biggest and best EVER cash give away!! These..',
 'Well done and ! luv ya all',
 'BIG BROTHER ALERT! The compute

In [13]:
# Load BERT preprocessing and encoding models from TensorFlow Hub
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)

In [14]:
# Define function to get BERT embeddings
def get_bert_embeddings(texts):
    text_input = tf.constant(texts)
    print(text_input)
    preprocessed_text = preprocessor(text_input)
    #print(preprocessed_text)
    outputs = encoder(preprocessed_text)
    #print(outputs)
    pooled_output = outputs['pooled_output']
    return pooled_output.numpy()

In [15]:
# Get BERT embeddings for training and test sets
train_embeddings = get_bert_embeddings(X_train.tolist())
test_embeddings = get_bert_embeddings(X_test.tolist())

tf.Tensor(
[b'Does daddy have a bb now.'
 b'Did you get any gift? This year i didnt get anything. So bad'
 b'Hello darling how are you today? I would love to have a chat, why dont you tell me what you look like and what you are in to sexy?'
 ...
 b'You have an important customer service announcement from PREMIER. Call FREEPHONE 0800 542 0578 now!'
 b'URGENT! Your Mobile number has been awarded with a \xc2\xa32000 prize GUARANTEED. Call 09061790126 from land line. Claim 3030. Valid 12hrs only 150ppm'
 b'How come it takes so little time for a child who is afraid of the dark to become a teenager who wants to stay out all night?'], shape=(1195,), dtype=string)
tf.Tensor(
[b'URGENT, IMPORTANT INFORMATION FOR O2 USER. TODAY IS YOUR LUCKY DAY! 2 FIND OUT WHY LOG ONTO HTTP://WWW.URAWINNER.COM THERE IS A FANTASTIC SURPRISE AWAITING FOR YOU'
 b'Panasonic & BluetoothHdset FREE. Nokia FREE. Motorola FREE & DoubleMins & DoubleTxt on Orange contract. Call MobileUpd8 on 08000839402 or call 2optout'
 

In [16]:
train_embeddings
train_embeddings.shape

(1195, 768)

In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [18]:
# Initialize and train the Random Forest model
rf_model = RandomForestClassifier()
rf_model.fit(train_embeddings, y_train)

# Predict on the test set
y_pred = rf_model.predict(test_embeddings)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.93
